In [ ]:
!rm -rf /content/drive


rm: cannot remove '/content/drive/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.shortcut-targets-by-id': Operation canceled
rm: cannot remove '/content/drive/.Trash-0': Directory not empty
rm: cannot remove '/content/drive/.Encrypted/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.Encrypted/.shortcut-targets-by-id': Operation canceled


Connect Google Drive to access input videos and save outputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

for root, dirs, files in os.walk("/content/drive/MyDrive"):
    if "PlayerTracking" in dirs:
        print("Found at:", os.path.join(root, "PlayerTracking"))
        break

Found at: /content/drive/MyDrive/PlayerTracking


In [ ]:
import os

print(os.listdir("/content/drive/MyDrive/PlayerTracking"))


['12081088_3840_2160_60fps.mp4', '18884230-hd_1920_1080_60fps.mp4', '14770541_3840_2160_60fps.mp4', '5837595-uhd_3840_2160_24fps.mp4', '12059268_3840_2160_60fps.mp4', '14770543_3840_2160_60fps.mp4', '19216260-hd_1920_1080_30fps.mp4', '20366524-uhd_3840_2160_30fps.mp4', 'frames_football1', 'frames', 'video_720p.mp4']


In [ ]:
path = "/content/drive/MyDrive/PlayerTracking"

if os.path.exists(path):
    print("✅ Folder exists:", path)
else:
    print("❌ Folder NOT found")


✅ Folder exists: /content/drive/MyDrive/PlayerTracking


In [ ]:
import os, glob

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))

print("Found videos:", len(videos))
for v in videos:
    print(" -", os.path.basename(v))


Found videos: 9
 - 12059268_3840_2160_60fps.mp4
 - 12081088_3840_2160_60fps.mp4
 - 14770541_3840_2160_60fps.mp4
 - 14770543_3840_2160_60fps.mp4
 - 18884230-hd_1920_1080_60fps.mp4
 - 19216260-hd_1920_1080_30fps.mp4
 - 20366524-uhd_3840_2160_30fps.mp4
 - 5837595-uhd_3840_2160_24fps.mp4
 - video_720p.mp4


Install YOLOv8 and OpenCV

In [ ]:
!pip -q install ultralytics opencv-python

from ultralytics import YOLO
print("✅ Setup OK")


✅ Setup OK


# **Player** Detection (YOLO) + Save Output Video

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")   # fast model

video_path = videos[0]       # first video
print("Using:", video_path)

model.predict(
    source=video_path,
    conf=0.4,
    save=True
)


Using: /content/drive/MyDrive/PlayerTracking/12059268_3840_2160_60fps.mp4

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/294) /content/drive/MyDrive/PlayerTracking/12059268_3840_2160_60fps.mp4: 384x640 8 persons, 150.2ms
video 1/1 (frame 2/294) /content/drive/MyDrive/PlayerTracking/12059268_3840_2160_60fps.mp4: 384x640 8 persons, 214.8ms
video 1/1 (frame 3/294) /content/drive/MyDrive/PlayerTracking/12059268_3840_2160_60fps.mp4: 384x640 8 persons, 175.7ms
video 1/1 (frame 4/294) /co

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

checks whether the detection results were saved

In [ ]:
import glob

print(glob.glob("runs/detect/*"))

['runs/detect/predict', 'runs/detect/predict2', 'runs/detect/predict3', 'runs/detect/predict4']


In [ ]:
print(glob.glob("runs/detect/predict*"))

['runs/detect/predict', 'runs/detect/predict2', 'runs/detect/predict3', 'runs/detect/predict4']


In [ ]:
import os

print(os.listdir("runs/detect/predict"))

['12059268_3840_2160_60fps.avi', 'detect_small.mp4']


displays the generated detection result video

In [ ]:
from IPython.display import Video

Video("runs/detect/predict/12059268_3840_2160_60fps.avi", embed=True)


Resize the video before displaying

In [ ]:
!ffmpeg -y -i "runs/detect/predict/12059268_3840_2160_60fps.avi" \
  -vf "scale=1280:720" \
  -vcodec libx264 -crf 28 \
  "runs/detect/predict/detect_small.mp4"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from IPython.display import Video
Video("runs/detect/predict/detect_small.mp4", embed=True)


Detection Model First - confusion Matrix

In [ ]:
import os

base = "/content/drive/MyDrive/PlayerTracking/dataset"
folders = [
    f"{base}/images/train",
    f"{base}/images/val",
    f"{base}/labels/train",
    f"{base}/labels/val",
]
for f in folders:
    os.makedirs(f, exist_ok=True)

print("✅ Dataset folders created:", base)



✅ Dataset folders created: /content/drive/MyDrive/PlayerTracking/dataset


In [ ]:
import cv2, glob, os

VIDEO_DIR = "/content/drive/MyDrive/PlayerTracking"
OUT_TRAIN = "/content/drive/MyDrive/PlayerTracking/dataset/images/train"
OUT_VAL   = "/content/drive/MyDrive/PlayerTracking/dataset/images/val"

videos = sorted(glob.glob(os.path.join(VIDEO_DIR, "*.mp4")))
print("Found videos:", len(videos))

save_every = 10   # save 1 frame every 10 frames (change if needed)
max_per_video = 120  # limit images per video

img_id = 0
for vi, vp in enumerate(videos):
    cap = cv2.VideoCapture(vp)
    frame_i = 0
    saved = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_i % save_every == 0:
            # 80% train, 20% val split
            out_dir = OUT_TRAIN if (img_id % 10) < 8 else OUT_VAL
            out_path = os.path.join(out_dir, f"img_{img_id:06d}.jpg")
            cv2.imwrite(out_path, frame)
            img_id += 1
            saved += 1
            if saved >= max_per_video:
                break

        frame_i += 1

    cap.release()
    print("✅ Extracted from:", os.path.basename(vp), "saved:", saved)

print("✅ Done. Total images:", img_id)


Found videos: 9
✅ Extracted from: 12059268_3840_2160_60fps.mp4 saved: 30
✅ Extracted from: 12081088_3840_2160_60fps.mp4 saved: 22
✅ Extracted from: 14770541_3840_2160_60fps.mp4 saved: 38
✅ Extracted from: 14770543_3840_2160_60fps.mp4 saved: 54
✅ Extracted from: 18884230-hd_1920_1080_60fps.mp4 saved: 29
✅ Extracted from: 19216260-hd_1920_1080_30fps.mp4 saved: 24
✅ Extracted from: 20366524-uhd_3840_2160_30fps.mp4 saved: 27
✅ Extracted from: 5837595-uhd_3840_2160_24fps.mp4 saved: 20
✅ Extracted from: video_720p.mp4 saved: 15
✅ Done. Total images: 259


In [ ]:
import os

BASE = "/content/drive/MyDrive/PlayerTracking/dataset"

folders = [
    f"{BASE}/labels/train",
    f"{BASE}/labels/val"
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("✅ Label folders created")


✅ Label folders created


In [ ]:
from ultralytics import YOLO
import glob
import os

model = YOLO("yolov8n.pt")

IMG_TRAIN = "/content/drive/MyDrive/PlayerTracking/dataset/images/train"
IMG_VAL   = "/content/drive/MyDrive/PlayerTracking/dataset/images/val"

LAB_TRAIN = "/content/drive/MyDrive/PlayerTracking/dataset/labels/train"
LAB_VAL   = "/content/drive/MyDrive/PlayerTracking/dataset/labels/val"

def auto_label(image_folder, label_folder):
    images = glob.glob(image_folder + "/*.jpg")
    print("Labelling:", len(images), "images")

    for img_path in images:
        results = model.predict(source=img_path, conf=0.4, verbose=False)

        txt_name = os.path.basename(img_path).replace(".jpg", ".txt")
        label_path = os.path.join(label_folder, txt_name)

        with open(label_path, "w") as f:
            for r in results:
                if r.boxes is not None:
                    for box in r.boxes:
                        cls = int(box.cls[0])
                        x, y, w, h = box.xywhn[0]
                        f.write(f"{cls} {x} {y} {w} {h}\n")

auto_label(IMG_TRAIN, LAB_TRAIN)
auto_label(IMG_VAL, LAB_VAL)

print("✅ Labels created")


Labelling: 208 images
Labelling: 51 images
✅ Labels created


In [ ]:
yaml_text = """
path: /content/drive/MyDrive/PlayerTracking/dataset
train: images/train
val: images/val

names:
  0: person
"""

with open("/content/drive/MyDrive/PlayerTracking/dataset/data.yaml", "w") as f:
    f.write(yaml_text)

print("✅ data.yaml created")


✅ data.yaml created


In [ ]:
!pip install -q ultralytics

from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="/content/drive/MyDrive/PlayerTracking/dataset/data.yaml",
    epochs=3,
    imgsz=640
)


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/PlayerTracking/dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bfc9ef1e870>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# **Player Tracking Using ByteTrack - IDs**

In [ ]:
model.track(
    source=video_path,
    conf=0.4,
    save=True,
    tracker="bytetrack.yaml"
)


Reload Video Path

In [ ]:
import glob
import os

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))

video_path = videos[0]   # choose first video
print("Using:", video_path)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.track(
    source=video_path,
    conf=0.4,
    save=True,
    tracker="bytetrack.yaml"
)



In [ ]:
import glob
print("Track folders:", glob.glob("runs/track/*"))
print("All track videos:", glob.glob("runs/track/**/*.*", recursive=True))


In [ ]:
import glob, os
track_vids = glob.glob("runs/track/**/*.avi", recursive=True) + glob.glob("runs/track/**/*.mp4", recursive=True)
print(track_vids)

One time Setup

Mount Google Drive + Install Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip -q install ultralytics opencv-python ffmpeg-python


In [ ]:
import os, glob

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))

print("Found videos:", len(videos))
for v in videos[:10]:
    print(" -", os.path.basename(v))

video_path = videos[0]
print("\n✅ Using:", video_path)


Selecting the Video from Google Drive

In [ ]:
import os, glob

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))

video_path = videos[0]
print("✅ Using:", video_path)



YOLO Player Detection

In [ ]:
from ultralytics import YOLO

det_model = YOLO("yolov8n.pt")

det_model.predict(
    source=video_path,
    conf=0.4,
    save=True,
    project="runs",
    name="detect_run"
)


Check Where YOLO Saved Result

In [ ]:
import os

print("Folders inside runs:")
print(os.listdir("runs"))




In [ ]:
import glob

print("All detect folders:")
print(glob.glob("runs/*"))


In [ ]:
import glob
print(glob.glob("runs/detect/*"))


In [ ]:
import os
print(os.listdir(glob.glob("runs/detect/predict*")[0]))


In [ ]:
import glob
print(glob.glob("runs/detect/predict*/*.avi"))
print(glob.glob("runs/detect/predict*/*.mp4"))


Set video_path (run once)

In [ ]:
import os, glob

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))
print("Found:", len(videos), "videos")

video_path = videos[0]
print("✅ Using:", video_path)


In [ ]:
small_vid = "/content/drive/MyDrive/PlayerTracking/video_720p.mp4"

!ffmpeg -y -i "{video_path}" -vf "scale=1280:720" -r 30 -vcodec libx264 -crf 28 "{small_vid}"

import os
print("Small video exists?", os.path.exists(small_vid))


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.track(
    source=small_vid,
    conf=0.4,
    save=True,
    tracker="bytetrack.yaml",
    project="runs",
    name="track_run",
    verbose=True
)


In [ ]:
import os
print("Track_run exists?", os.path.exists("runs/track_run"))


In [ ]:
import glob

track_dirs = glob.glob("runs/**/track_run*", recursive=True)
print("Found track dirs:", track_dirs)


In [ ]:
track_dir = track_dirs[-1]
track_vids = glob.glob(track_dir + "/*.avi") + glob.glob(track_dir + "/*.mp4")
print(track_vids)

track_out = track_vids[0]
print("✅ Using tracking output:", track_out)



In [ ]:
!ffmpeg -y -i "{track_out}" -vf "scale=1280:720" -vcodec libx264 -crf 28 "{track_dir}/tracking_small.mp4"


In [ ]:
from IPython.display import Video
Video(f"{track_dir}/tracking_small.mp4", embed=True)


# **Pose / Keypoint Detection**

In [ ]:
from ultralytics import YOLO

pose_model = YOLO("yolov8n-pose.pt")

pose_model.predict(
    source=small_vid,
    conf=0.3,
    save=True,
    project="runs",
    name="pose_run",
    verbose=True
)


Locate Pose Output Folder and Video

In [ ]:
import glob, os

pose_dirs = sorted(glob.glob("runs/**/pose_run*", recursive=True))
print("Pose dirs found:", pose_dirs)

pose_dir = pose_dirs[-1]   # use the latest one
print("✅ Using pose_dir:", pose_dir)

pose_vids = glob.glob(pose_dir + "/*.avi") + glob.glob(pose_dir + "/*.mp4")
print("Pose videos:", pose_vids)


In [ ]:
pose_out = pose_vids[0]
print("✅ Pose output:", pose_out)

!ffmpeg -y -i "{pose_out}" -vf "scale=1280:720" -vcodec libx264 -crf 28 "{pose_dir}/pose_small.mp4"

print("✅ Made:", f"{pose_dir}/pose_small.mp4")


Display Pose Result

In [ ]:
from IPython.display import Video
Video("runs/pose/runs/pose_run/pose_small.mp4", embed=True)


In [ ]:
import os

print(os.listdir("/content"))





In [ ]:
print(os.listdir("/content/runs") if os.path.exists("/content/runs") else "No runs folder")


# **Testing The model**

In [ ]:
import os

base = "/content/drive/MyDrive/PlayerTracking/dataset"
folders = [
    f"{base}/images/train",
    f"{base}/images/val",
    f"{base}/labels/train",
    f"{base}/labels/val",
]
for f in folders:
    os.makedirs(f, exist_ok=True)

print("✅ Dataset folders created:", base)


In [ ]:
VIDEO_DIR = "/content/drive/MyDrive/PlayerTracking"
OUT_TRAIN = "/content/drive/MyDrive/PlayerTracking/dataset/images/train"
OUT_VAL   = "/content/drive/MyDrive/PlayerTracking/dataset/images/val"

videos = sorted(glob.glob(os.path.join(VIDEO_DIR, "*.mp4")))
print("Found videos:", len(videos))

save_every = 10   # save 1 frame every 10 frames (change if needed)
max_per_video = 120  # limit images per video

img_id = 0
for vi, vp in enumerate(videos):
    cap = cv2.VideoCapture(vp)
    frame_i = 0
    saved = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_i % save_every == 0:
            # 80% train, 20% val split
            out_dir = OUT_TRAIN if (img_id % 10) < 8 else OUT_VAL
            out_path = os.path.join(out_dir, f"img_{img_id:06d}.jpg")
            cv2.imwrite(out_path, frame)
            img_id += 1
            saved += 1
            if saved >= max_per_video:
                break

        frame_i += 1

    cap.release()
    print("✅ Extracted from:", os.path.basename(vp), "saved:", saved)

print("✅ Done. Total images:", img_id)

In [ ]:
import os

BASE = "/content/drive/MyDrive/PlayerTracking/dataset"

folders = [
    f"{BASE}/labels/train",
    f"{BASE}/labels/val"
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("✅ Label folders created")

In [ ]:
from ultralytics import YOLO
import glob
import os

model = YOLO("yolov8n.pt")

IMG_TRAIN = "/content/drive/MyDrive/PlayerTracking/dataset/images/train"
IMG_VAL   = "/content/drive/MyDrive/PlayerTracking/dataset/images/val"

LAB_TRAIN = "/content/drive/MyDrive/PlayerTracking/dataset/labels/train"
LAB_VAL   = "/content/drive/MyDrive/PlayerTracking/dataset/labels/val"

def auto_label(image_folder, label_folder):
    images = glob.glob(image_folder + "/*.jpg")
    print("Labelling:", len(images), "images")

    for img_path in images:
        results = model.predict(source=img_path, conf=0.4, verbose=False)

        txt_name = os.path.basename(img_path).replace(".jpg", ".txt")
        label_path = os.path.join(label_folder, txt_name)

        with open(label_path, "w") as f:
            for r in results:
                if r.boxes is not None:
                    for box in r.boxes:
                        cls = int(box.cls[0])
                        x, y, w, h = box.xywhn[0]
                        f.write(f"{cls} {x} {y} {w} {h}\n")

auto_label(IMG_TRAIN, LAB_TRAIN)
auto_label(IMG_VAL, LAB_VAL)

print("✅ Labels created")


In [ ]:
yaml_text = """
path: /content/drive/MyDrive/PlayerTracking/dataset
train: images/train
val: images/val

names:
  0: person
"""

with open("/content/drive/MyDrive/PlayerTracking/dataset/data.yaml", "w") as f:
    f.write(yaml_text)

print("✅ data.yaml created")

In [ ]:
!pip install -q ultralytics

from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="/content/drive/MyDrive/PlayerTracking/dataset/data.yaml",
    epochs=3,
    imgsz=640
)